In [19]:
#uncomment this line if you want to install package
!pip install -r requirements.txt
#!pip install fastapi
#pip install --upgrade pip
#print("Hello world")

  Using cached kafka_python-2.0.2-py2.py3-none-any.whl (246 kB)


In [25]:
!pip uninstall confluent_kafka

^C


In [24]:
import selenium.common.exceptions
import urllib3.exceptions
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from time import sleep
import threading
import csv
import traceback
import multiprocessing
from typing import Union
from fastapi import FastAPI
import json
from n_kafka.producer import ProjectProducer


ImportError: cannot import name 'IncompatibleBrokerVersion' from 'kafka.errors' (e:\software\anaconda\lib\site-packages\kafka\errors.py)

In [7]:
def split_list(lst, k):
    """
    This function is used to split a list to sub-lists that have k items.

    Args:
        lst (list): input list
        k (int): number of item of a sub-list

    Returns:
        list: list of sub-list
    For example:
    split_list([1,2,3,4,5],2) => [[1,2],[3,4],[5]]
    """
    size = len(lst)
    return [lst[i:i+k] for i in range(0, size, k)]


print(split_list(["hi", 2, 3, 4, 5], 3))
print([1,2,3,4,5,6][:-1])


[['hi', 2, 3], [4, 5]]
[1, 2, 3, 4, 5]


In [8]:
def get_first_not_null_item(lsts):
    """
      this function is used to get the first not null item of a list, it there is no not-null item,
      this func returns the last item
      
    Args:
        lsts (list): input list

    Returns:
        string: the first not null item
    """
    for l in lsts:
        if (len(l)!=0):
            return l
    return lsts[-1]
print(get_first_not_null_item(['','1','']))

1


In [12]:
def get_detail_project(page, url,error_url_file,producer_info=[]):
    """
    This func is used to crawl data of a detail project page of Kickstarter website

    Args:
        page (int): page index of url
        url (string): link to website that need to be crawled
        error_url_file (string): name of file that contains a list of error urls that can not be crawled (
            this list will be executed later
        )
    """
    browser = webdriver.Chrome(
        service=(Service(ChromeDriverManager().install()))
    )
    try:
        print("crawl url: ")
        print(url)
        browser.get(url)
        # sleep(2)
        # t = title.text
        wait = WebDriverWait(browser, 10)
        title = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "h2.type-24-md.soft-black.mb1.project-name"))))
        description = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "p[class='type-14 type-18-md soft-black project-description mb1']"))))
        picture = browser.find_element(
            By.CSS_SELECTOR, "img[class='aspect-ratio--object bg-black z3']").get_attribute("src")
        pledged = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "span[class='ksr-green-500']"))))
        span_goal = browser.find_elements(By.CSS_SELECTOR, "span[class='inline-block-sm hide']")
        n_goal = []
        for i in span_goal:
            goals = i.find_elements(By.CSS_SELECTOR, "span[class='money']")
            for goal in goals:
                n_goal.append(goal.text)
        goal = get_first_not_null_item(n_goal)
        num_of_backer = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "div[class='block type-16 type-28-md bold dark-grey-500']"))))
        days_to_go = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR, "span[class='block type-16 type-28-md bold dark-grey-500']"))))
        mark_field_locations = list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR,"a.nowrap.navy-700.flex.items-center.medium.mr3.type-12.keyboard-focusable > span.ml1"
        )))
        mark = mark_field_locations[3]
        field = mark_field_locations[4]
        location = mark_field_locations[5]
        num_of_comment = get_first_not_null_item(list(map(lambda a:a.text,browser.find_elements(
            By.CSS_SELECTOR,
            "a.js-analytics-section.js-load-project-comments.js-load-project-content.mx3.project-nav__link--comments.tabbed-nav__link.type-14 > span.count"
        ))))
        res = {
            "title":title,
            "description":description,
            "picture":picture,
            "pledged":pledged,
            "goal":goal,
            "num_of_backer":num_of_backer,
            "days_to_go":days_to_go,
            "mark":mark,
            "field":field,
            "location":location,
            "num_of_comment":num_of_comment
        }
        if (len(producer_info)>0):
            broker,topic = producer_info
            print("[*] Sending to broker "+broker+", topic: "+topic)
            print(json.dumps(res).encode("utf-8"))
            projectProducer = ProjectProducer(broker=broker)
            try:
                projectProducer.send_msg_sync(json.dumps(res).encode("utf-8"), topic=topic)
                print("[*] Send to kafka successfully.")
            except:
                print("[*] Send to kafka fail.") 
        else:
        #  file = open("./data/result.txt","a")
        #  file.write(url+"\n")
        #  file.write(str(res)+"\n")
        #  file.close()
          print("[*] Save data to mongodb.")
    except:
        error_url = str(page)+","+url+"\n"
        error_url_file_obj = open(error_url_file, "a")
        error_url_file_obj.write(error_url)
        error_url_file_obj.close()
        traceback.print_exc()
    browser.close()
# get_data(current_page=current_page)
get_detail_project(0,"https://www.kickstarter.com/projects/mlspencer/dragon-mage-deluxe-collectors-edition-hardcover",error_url_file="./data/error_url.csv",producer_info=["localhost:9092","kickstarter_project"])


crawl url: 
https://www.kickstarter.com/projects/mlspencer/dragon-mage-deluxe-collectors-edition-hardcover
[*] Sending to broker localhost:9092, topic: kickstarter_project
b'{"title": "Dragon Mage: Deluxe Collector\'s Edition Hardcover", "description": "A deluxe, signed, faux-leather Collector\'s Edition of the first book in the multi-award-winning epic fantasy series Rivenworld.", "picture": "https://ksr-ugc.imgix.net/assets/041/284/528/eaed1a1d708b34bbc525d42fe063f646_original.png?ixlib=rb-4.0.2&crop=faces&w=1024&h=576&fit=crop&v=1686684665&auto=format&frame=1&q=92&s=2a9491a0399da60e0010d30f6dde643d", "pledged": "US$ 60,607", "goal": "US$ 14,000", "num_of_backer": "507", "days_to_go": "14", "mark": "Project We Love", "field": "Fiction", "location": "Redlands, CA", "num_of_comment": "55"}'
Send message synchronously
Message delivery failed: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
[*] Send to kafka successfully.


In [14]:
broker,topic = "localhost:9092","kickstarter_project"
print("[*] Sending to broker "+broker+", topic: "+topic)
msg = "Oh my gosh."
print(msg)
projectProducer = ProjectProducer(broker=broker)
try:
    projectProducer.send_msg_sync(msg, topic=topic)
    print("[*] Send to kafka successfully.")
except:
    print("[*] Send to kafka fail.") 

[*] Sending to broker 127.0.0.1:9092, topic: kickstarter_project
Oh my gosh.
Send message synchronously
Message delivery failed: KafkaError{code=_MSG_TIMED_OUT,val=-192,str="Local: Message timed out"}
[*] Send to kafka successfully.


In [31]:
def get_data(url,current_page,num_of_thread,error_url_file,checkpoint_file):
    """
    This func is used to crawl data from Kickstarter website (
        https://www.kickstarter.com/discover/advanced?woe_id=0&sort=magic&seed=2811224&page=
    )

    Args:
        url (string): link to website
        current_page (int): current page index of page that is being crawled
        num_of_thread (int): num of crawling thread 
        error_url_file (string): name of file that contains a list of error_url_file that can not be crawled
        checkpoint_file (string): name of file that contains information about the index of page that is being crawled
    """
    page = current_page
    while (1):
        browser = webdriver.Chrome(
        service=(Service(ChromeDriverManager().install()))
    )
        meta_url = url+str(page)
        print("meta_url: ")
        print(meta_url)
        try:
            browser.get(meta_url)
            sleep(2)
            links = list(map(lambda a: a.get_attribute("href"),
                                 browser.find_elements(By.CSS_SELECTOR,
                                                       "a[class='block img-placeholder w100p']")
                                 ))
            prj_links = [l for l in links if l.endswith("?ref=discovery")]
            print("prj_links: ")
            for l in prj_links:
                print(l)
            print(len(prj_links))
            threads = []
            split_prj_links = split_list(prj_links, num_of_thread)
            last_prj_links = split_prj_links[-1]
            print("split_prj_links: ")
            print(split_prj_links)
            for links in split_prj_links[:-1]:
                threads = [threading.Thread(
                        target=get_detail_project, args=(current_page, link,error_url_file)) for link in links]
                for thread in threads:
                    thread.start()
                for thread in threads:
                    thread.join()
                threads = []
            threads = [threading.Thread(
                    target=get_detail_project, args=(current_page, link,error_url_file)) for link in last_prj_links]
            for thread in threads:
                thread.start()
            for thread in threads:
                thread.join()
            threads = []
            page = page+1
        except:
            file = open(checkpoint_file, "w")
            file.write(str({"page": page}))
            traceback.print_exc()
            break
        browser.close()


In [33]:
# link to the website Kickstarter
url = 'https://www.kickstarter.com/discover/advanced?woe_id=0&sort=magic&seed=2811224&page='

# get the current page
checkpoint = eval(open('./data/checkpoint.csv', "r").readline())
current_page = checkpoint["page"]
error_url_file = 'error_url.csv'
print(current_page)

# start to crawl
app = FastAPI()
chrome_options = Options()
chrome_options.add_experimental_option('detach',True)
@app.get("/projects")
def get_data_from_kickstarter(num_of_thread:int = 4):
 get_data(current_page=current_page,url=url,num_of_thread=num_of_thread,checkpoint_file="./data/checkpoint.csv",error_url_file="./data/error_url.csv")


0


### test some function

In [7]:
print("geats")
browser = webdriver.Chrome(
        service=(Service(ChromeDriverManager().install()))
    )
meta_url = url+str(4)
browser.get(meta_url)
sleep(1)
browser.close()
browser = webdriver.Chrome(
        service=(Service(ChromeDriverManager().install()))
    )
browser.get(url+str(5))
sleep(1)
# get_data(current_page=current_page)
# get_detail_project(0,"https://www.kickstarter.com/projects/mlspencer/dragon-mage-deluxe-collectors-edition-hardcover")


geats


In [8]:
# f = open(file,"a")
# f.writelines("hello")
# f.writelines("hi")
# f.close()
# f = open(file,"r")
# l = list(map(lambda a:a.strip().split(","),f.readlines()))
# print(l)
print("hello")
x = ''
print(len(x))
def task(x):
    print("multi thread")
    print(x * x)
data = [[1,2],[2,3],[4,5]]
inputs = [100,2,3,4,5,7]

res = [i for i in inputs if i % 2==0]
print(res)
threads =[]
# for m in inputs:
#     thread = threading.Thread(target=task,args=[m])
#     threads.append(thread)
threads = [threading.Thread(target=task,args=[m]) for m in inputs]
for thread in threads:
    thread.start()
    thread.join()
# pool.close()
#pool.join()
#print("results: {}".format(results))


hello
0
[100, 2, 4]
multi thread
10000
multi thread
4
multi thread
9
multi thread
16
multi thread
25
multi thread
49


In [9]:
print("Hello world")
for i in range(0, 5):
    print(i)


Hello world
0
1
2
3
4


In [7]:
x = None
y = {
    "x": x if x is not None else 2
}
print(y)
m = [1,2,3]
a,b,c = m
print(a,b,c)

{'x': 2}
1 2 3
